In [24]:
import numpy as np
import math

class Vector(object):
    
    CANNOT_NORMALISE_ZERO_VECTOR_MSG = 'Cannot normalise the zero vector'
    
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple(coordinates)
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')


    def __str__(self):
        return 'Vector: {}'.format(self.coordinates)


    def __eq__(self, v):
        return self.coordinates == v.coordinates
    
    def __add__(self, v):
        if self.dimension == v.dimension :
            return [x + y for x, y in zip(self.coordinates, v.coordinates)]
        else :
            return
        
    def __sub__(self, v):
        if self.dimension == v.dimension :
            return [x - y for x, y in zip(self.coordinates, v.coordinates)]
        else :
            return
    
    def scalar_mul(self, s):
        return [s*x for x in self.coordinates]
    
    def magnitude(self):
        return sum([x**2 for x in self.coordinates])**(.5)
    
    def normalise(self):
        try :
            return [(1./self.magnitude())*x for x in self.coordinates]
        except ZeroDivisionError:
            raise Exception('Cannot normalise zero vector')
        
    def dot(self, v):
        return sum([x*y for x, y in zip(self.coordinates, v.coordinates)])
    
    def angle(self, v, in_degrees=False):
        if in_degrees:
            return math.degrees(np.arccos(self.dot(v)/(self.magnitude() * v.magnitude())))
        else:    
            return np.arccos(round(self.dot(v)/(self.magnitude() * v.magnitude()),10))
#         try :
#             u1 = self.normalise()
#             u2 = v.normalise()
#             angle_in_radians = math.acos(u1.dot(u2))
            
#             if in_degrees:
#                 degrees_per_radian = 180. / math.pi
#                 return angle_in_radians * degrees_per_radian
#             else:
#                 return angle_in_radians
            
#         except Exception as e:
#             if str(e) == self.CANNOT_NORMALISE_ZERO_VECTOR_MSG:
#                 raise Exception('Cannot computer an angle with the zero vector')
#             else:
#                 raise e

    def orthogonal(self, v, tolerance=1e-10):
        return abs(self.dot(v)) < tolerance

    def is_zero(self, tolerance=1e-10):
        return self.magnitude() < tolerance

    def parallel(self, v):
        return (self.is_zero() or v.is_zero() or self.angle(v)==0 or self.angle(v)==math.pi)
    
    def projection(self, b):
        return Vector(b.normalise()).scalar_mul(self.dot(Vector(b.normalise())))
    
    def component_orthogonal_to(self, b):
        projection = Vector(self.projection(b))
        return self - projection
    
    def cross_product(self, v):
        a = self.coordinates
        b = v.coordinates    
        if  self.dimension == 2 and v.dimension == 2:
            # add 0 for the z dimension
            a = a + (0,)
            b = b + (0,)
        
        return [ a[1]*b[2] - b[1]*a[2], -(a[0]*b[2] - b[0]*a[2]), a[0]*b[1] - b[0]*a[1] ]
    
    def parallelogram_area(self, v):
        return sum([ x**2 for x in self.cross_product(v)])**(1./2)
    
    def triangle_area(self, v):
        return self.parallelogram_area(v)*(1./2)
    
    def __getitem__(self, i):
        return self.coordinates[i]

    def __iter__(self):
        return self.coordinates.__iter__()

In [25]:
from decimal import Decimal, getcontext

getcontext().prec = 30

class Plane(object):

    NO_NONZERO_ELTS_FOUND_MSG = 'No nonzero elements found'

    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 3

        if not normal_vector:
            all_zeros = [0]*self.dimension
            normal_vector = Vector(all_zeros)
        self.normal_vector = Vector(normal_vector)

        if not constant_term:
            constant_term = 0
        self.constant_term = constant_term

        self.set_basepoint()


    def set_basepoint(self):
        try:
            n = self.normal_vector
            c = self.constant_term
            basepoint_coords = [0]*self.dimension

            initial_index = Plane.first_nonzero_index(n)
            initial_coefficient = n[initial_index]

            basepoint_coords[initial_index] = c/initial_coefficient
            self.basepoint = Vector(basepoint_coords)

        except Exception as e:
            if str(e) == Plane.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                raise e


    def __str__(self):

        num_decimal_places = 3

        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)

            output = ''

            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'

            if not is_initial_term:
                output += ' '

            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))

            return output

        n = self.normal_vector

        try:
            initial_index = Plane.first_nonzero_index(n)
            terms = [write_coefficient(n[i], is_initial_term=(i==initial_index)) + 'x_{}'.format(i+1)
                     for i in range(self.dimension) if round(n[i], num_decimal_places) != 0]
            output = ' '.join(terms)

        except Exception as e:
            if str(e) == self.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e

        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)

        return output


    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Plane.NO_NONZERO_ELTS_FOUND_MSG)
        
    def is_parallel_to(self, p):
        # parallel planes have parallel normal vectors
        n1 = self.normal_vector
        n2 = p.normal_vector
        
        return n1.parallel(n2)
    
    def __eq__(self, p):
        if not self.is_parallel_to(p):
            return False

        x = self.basepoint
        y = p.basepoint
        diff = Vector(x - y)
        n = self.normal_vector

        if diff.orthogonal(n):
            return True
        else:
            return False


class MyDecimal(Decimal):
    def is_near_zero(self, eps=1e-10):
        return abs(self) < eps

In [26]:
p1 = Plane([-0.412, 3.806, 0.728], -3.46)
p2 = Plane([1.03, -9.515, -1.82], 8.65)
p3 = Plane([2.611, 5.528, 0.283], 4.6)
p4 = Plane([7.715, 8.306, 5.342], 3.76)
p5 = Plane([-7.926, 8.625, -7.212], -7.952)
p6 = Plane([-2.642, 2.875, -2.404], -2.443)

print 'p1', p1
print 'p2', p2
#print('p1 :', p1)
#print('p2 :', p2)
print(p1.is_parallel_to(p2))
print(p1 == p2)

print 'p3', p3
print 'p4', p4
#print('p3 :', p3)
#print('p4 :', p4)
print(p3.is_parallel_to(p4))
print(p3 == p4)

print 'p5', p5
print 'p6', p6
#print('p5 :', p5)
#print('p6 :', p6)
print(p5.is_parallel_to(p6))
print(p5 == p6)

p1 -0.412x_1 + 3.806x_2 + 0.728x_3 = -3.46
p2 1.03x_1 - 9.515x_2 - 1.82x_3 = 8.65
True
True
p3 2.611x_1 + 5.528x_2 + 0.283x_3 = 4.6
p4 7.715x_1 + 8.306x_2 + 5.342x_3 = 3.76
False
False
p5 -7.926x_1 + 8.625x_2 - 7.212x_3 = -7.952
p6 -2.642x_1 + 2.875x_2 - 2.404x_3 = -2.443
True
False
